In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [2]:
df_ts = pd.read_csv('https://raw.githubusercontent.com/OPERA-Cal-Val/DIST-Validation/refs/heads/main/referenceTimeSeries_last.csv')
df_ts.head()

,ID,overallLabel,Long,Lat,changetype,20211001,20211002,20211003,20211004,20211005,...,20220921,20220922,20220923,20220924,20220925,20220926,20220927,20220928,20220929,20220930
0,1,VLmaj,-52.379792,-23.297147,Natural browning,noObs,noObs,noObs,noObs,noObs,...,noChange,noChange,noChange,noObs,noObs,noObs,noObs,noObs,noObs,noObs
1,2,noChange,129.850328,-29.577240,No Change,noObs,noObs,noObs,noChange,noChange,...,noChange,noChange,noChange,noChange,noObs,noObs,noObs,noObs,noObs,noObs
2,3,VLmin,-98.600559,33.771004,Natural browning,noObs,noObs,noChange,noChange,noChange,...,noChange,noChange,noChange,noChange,noObs,noObs,noObs,noObs,noObs,noObs
3,4,VLmaj,19.332894,4.574903,Fire,noObs,noObs,noChange,noChange,noChange,...,noChange,noChange,noChange,noChange,noChange,noChange,noChange,noObs,noObs,noObs
4,5,VLmaj,115.619006,63.108089,Fire,noObs,noObs,noObs,noObs,noObs,...,noChange,noChange,noChange,noChange,noObs,noObs,noObs,noObs,noObs,noObs


# Break Apart TS by Site

In [3]:
records = df_ts.to_dict('records')

def format_row(record: dict) -> dict:
    out =[]
    for key in record.keys():
        if key[:2] == '20':
            item = {'site_id': record['ID'],
                    'date': pd.Timestamp(key),
                    'obs': record[key]}
            out.append(item)
    return out
    

In [4]:
df_example = pd.DataFrame(format_row(records[0]))
df_example.head()

,site_id,date,obs
0,1,2021-10-01,noObs
1,1,2021-10-02,noObs
2,1,2021-10-03,noObs
3,1,2021-10-04,noObs
4,1,2021-10-05,noObs


In [5]:
data = list(map(format_row, records))
dfs = list(map(pd.DataFrame, data))

In [6]:
dfs[10].head()

,site_id,date,obs
0,11,2021-10-01,noObs
1,11,2021-10-02,noObs
2,11,2021-10-03,noObs
3,11,2021-10-04,noObs
4,11,2021-10-05,noChange


# Serialize

In [7]:
ts_dir = Path('val_timeseries_by_site')
ts_dir.mkdir(exist_ok=True)

In [8]:
_ = [df.to_csv(ts_dir / f'site_{site_id}.csv', index=False) for (site_id, df) in zip(df_ts.ID.tolist(), dfs)]